# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 4: Web Scraping Job Postings

## Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal has two main objectives:

   1. Determine the industry factors that are most important in predicting the salary amounts for these data.
   2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Seek.com](https://www.seek.com) regularly pool job postings from a variety of markets and industries. 

**Goal:** Scrape your own data from a job aggregation tool like seek.com in order to collect the data to best answer these two questions.

---

## Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to answer the two questions described above.

### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

### QUESTION 2: Factors that distinguish job category

Using the job postings you scraped for part 1 (or potentially new job postings from a second round of scraping), identify features in the data related to job postings that can distinguish job titles from each other. There are a variety of interesting ways you can frame the target variable, for example:
- What components of a job posting distinguish data scientists from other data jobs?
- What features are important for distinguishing junior vs. senior positions?
- Do the requirements for titles vary significantly with industry (e.g. healthcare vs. government)?

You may end up making multiple classification models to tackle different questions. Be sure to clearly explain your hypotheses and framing, any feature engineering, and what your target variables are. The type of classification model you choose is up to you. Be sure to interpret your results and evaluate your models' performance.
 
## Requirements

1. Scrape and prepare your own data.

2. **Create and compare at least two models for each section**. One of the two models should be a decision tree or ensemble model. The other can be a classifier or regression of your choosing (e.g. Ridge, logistic regression, KNN, SVM, etc).
   - Section 1: Job Salary Trends
   - Section 2: Job Category Factors

3. Prepare a polished Jupyter Notebook with your analysis for a peer audience of data scientists. 
   - Make sure to clearly describe and label each section.
   - Comment on your code so that others could, in theory, replicate your work.

4. A brief writeup in an executive summary, written for a non-technical audience.
   - Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

 

## Suggestions for Getting Started

1. Collect data from [seek.com](www.seek.com) (or another aggregator) on data-related jobs to use in predicting salary trends for your analysis.
  - Select and parse data from *at least 1000 postings* for jobs, potentially from multiple location searches.
2. Find out what factors most directly impact salaries (e.g. title, location, department, etc).
  - Test, validate, and describe your models. What factors predict salary category? How do your models perform?
3. Discover which features have the greatest importance when determining a low vs. high paying job.
  - Your Boss is interested in what overall features hold the greatest significance.
  - HR is interested in which SKILLS and KEY WORDS hold the greatest significance.   
4. Author an executive summary that details the highlights of your analysis for a non-technical audience.
 
---

## Useful Resources

- Scraping is one of the most fun, useful and interesting skills out there. Don’t lose out by copying someone else's code!
- [Here is some advice on how to write for a non-technical audience](http://programmers.stackexchange.com/questions/11523/explaining-technical-things-to-non-technical-people)
- [Documentation for BeautifulSoup can be found here](http://www.crummy.com/software/BeautifulSoup/).

---

### Project Feedback + Evaluation

For all projects, students will be evaluated on a simple 4 point scale (0, 1, 2 or 3). Instructors will use this rubric when scoring student performance on each of the core project **requirements:** 

 Score | Expectations
 ----- | ------------
 **0** | _Did not complete. Try again._
 **1** | _Does not meet expectations. Try again_
 **2** | _Meets expectations._
 **3** | _Surpasses expectations. Brilliant!_
 
 # Project 4 feedback
| Requirement | Rubric   |
|------|------|
|   Scrape and prepare your own data  | |
|   Create and compare at least two models, one a decision tree or ensemble and the other a classifier or regression for Section 1: Job Salary Trends and Section 2: Job Category Factors (so at least 4 models in total)  | |
|   Polished Jupyter notebook with your analysis annotated for a peer audience of data scientists  | |
|  Executive summary at the beginning of your notebook for written for your superiors to use to make business decisions. Make sure that it includes the ‘So what…’ regarding your analysis, risks and limitations. ||
|   
__Qualitative feedback:__


In [2]:
import json 

import requests

import numpy as np
from scrapy.selector import Selector
from bs4 import BeautifulSoup # From Sri Class on Sat 15/01/2021
import pandas as pd
import matplotlib.pyplot as plt

import ast # From Geoff

In [7]:
data_DA_json = pd.read_json('data-DA.json')
data_bi_json = pd.read_json('data-bi.json')
data = pd.concat([data_DA_json, data_bi_json])
data

,Job Title,Company,multiple_details,Description
0,Data Scientist,Fundo Loans,"[Sydney, CBD, Inner West & Eastern Suburbs, In...",We are scaling up! An exciting opportunity exi...
1,Data Analyst,AIA Australia Limited,"[Melbourne, CBD & Inner Suburbs, Banking & Fin...",The focus of this role is to provide support t...
2,Junior Business Intelligence Analyst,Hays Talent Solutions,"[Sydney, CBD, Inner West & Eastern Suburbs, In...","Your new companyAt Hays, we are on a journey t..."
3,Data Analyst,Atos Australia,"[Sydney, CBD, Inner West & Eastern Suburbs, In...",About Atos:Atos is a global leader in digital ...
4,Data Analyst,Infrastructure Partnerships Australia,"[Sydney, CBD, Inner West & Eastern Suburbs, Co...",About Infrastructure Partnerships AustraliaInf...
...,...,...,...,...
534,Data Analyst,Aurec,"[ACT, Information & Communication Technology, ...",We are looking to engage a skilled and enthusi...
535,Senior Data Analyst - NSW Government,Talenza,"[Sydney, CBD, Inner West & Eastern Suburbs, In...",Senior Data Analyst - NSW GovernmentLocation: ...
536,Principal Engineer (Knowledge Graph),SEEK Limited,"[Melbourne, CBD & Inner Suburbs, Information &...",Company DescriptionAbout SEEKSEEK’s portfolio ...
537,Data Management Analyst,Humanised Group,"[Brisbane, CBD & Inner Suburbs, Information & ...",About the role:In this role you will be requir...


In [9]:
# string form of list
data['multiple_details'].head().values

array([list(['Sydney', 'CBD, Inner West & Eastern Suburbs', 'Information & Communication Technology', 'Engineering - Software', '$110,000 - $149,999', 'Full time']),
       list(['Melbourne', 'CBD & Inner Suburbs', 'Banking & Financial Services', 'Analysis & Reporting', 'Full time']),
       list(['Sydney', 'CBD, Inner West & Eastern Suburbs', 'Information & Communication Technology', 'Business/Systems Analysts', 'Full time']),
       list(['Sydney', 'CBD, Inner West & Eastern Suburbs', 'Information & Communication Technology', 'Business/Systems Analysts', 'Full time']),
       list(['Sydney', 'CBD, Inner West & Eastern Suburbs', 'Consulting & Strategy', 'Analysts', 'Full time'])],
      dtype=object)

In [11]:
# parse string into list using ast
import ast
data['multiple_details_lst'] = data.multiple_details.apply(lambda x: ast.literal_eval(x))

ValueError: malformed node or string: ['Sydney', 'CBD, Inner West & Eastern Suburbs', 'Information & Communication Technology', 'Engineering - Software', '$110,000 - $149,999', 'Full time']